#### Das Verzeichnis festlegen

In [1]:
import sys
from pathlib import Path

# Zum übergeordneten Ordner 'src' wechseln
current_path = Path().resolve()
src_path = current_path.parents[2]  # Zwei Stufen nach oben

if str(src_path) not in sys.path:
    sys.path.append(str(src_path))

print(f"Achtung, der globale Pfad wurde auf {src_path} gesetzt")


Achtung, der globale Pfad wurde auf C:\Users\Nasiba\Documents\1 Master Data Science\Projektpraktikum\WebScience24 gesetzt


#### Imports & Downloads

In [ ]:
!pip install emoji
!pip install pyspellchecker
!pip install textblob
!pip install ftfy
!python -m spacy download en_core_web_sm
!pip install ekphrasis

In [2]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
import re
import emoji
from nltk.corpus import stopwords, wordnet
import time
from pathlib import Path

from collections import Counter
from spellchecker import SpellChecker
import os

from textblob import TextBlob, Word
from nltk.stem.snowball import SnowballStemmer, PorterStemmer
from nltk.stem import WordNetLemmatizer
from ftfy import fix_encoding
import spacy


from src.functions.clean_data_generic_functions import clean_misspelled_words, correct_misspelled_words_in_sentence,create_word_counter,expand_slang,expand_shortcuts,extract_emojis, to_lowercase, to_lowercase_if_string, handle_hashtags, handle_userhandles, lemmatize, remove_word_from_column, remove_na_from_column, replace_text_smileys, segment_tweets
from src.functions.clean_data_generic_functions import remove_special_characters, remove_digits, remove_duplicates, remove_emoji_in_sentence, remove_emojis, remove_freqwords,remove_hashtag_sign_from_tweet,remove_least_frequent_words, remove_most_frequent_words, remove_punctuation, remove_stop_words, remove_url_from_tweet, replace_emoji_in_sentence, replace_emojis


nlp = spacy.load("en_core_web_sm")

pd.set_option('display.max_colwidth', None)

Reading english - 1grams ...
Reading english - 2grams ...


C:\Users\Nasiba\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\ekphrasis\classes\exmanager.py:14: FutureWarning: Possible nested set at position 42
  regexes = {k.lower(): re.compile(self.expressions[k]) for k, v in


Reading english - 1grams ...
Reading english - 2grams ...
Reading twitter - 1grams ...
Reading twitter - 2grams ...


In [3]:
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger_eng')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Nasiba\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Nasiba\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Nasiba\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Nasiba\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     C:\Users\Nasiba\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


True

#### Beladung der Daten

#### Trainingsdaten

In [4]:

source_filepath_name = os.path.abspath(os.path.join(src_path, 'data/twitter_hate-speech/train.csv'))

df_origin = pd.read_csv(source_filepath_name, encoding='utf-8', index_col=0)

#### Testdaten

In [5]:
source_filepath_name_test = os.path.abspath(os.path.join(src_path, 'data/twitter_hate-speech/test.csv'))
df_origin_test = pd.read_csv(source_filepath_name_test, encoding='utf-8', index_col=0)

### Datenvorbereitung

In [6]:
df_clean_base = df_origin.copy()
df_clean_base_test = df_origin_test.copy()

#### 1. Datenbereinigung allgemein

##### Duplikatenentfernung

In [7]:
df_clean_base.drop_duplicates(inplace=True)

df_clean_base.duplicated().sum()



0

#### Encoding

In [8]:
df_clean_base["tweet"] = df_clean_base['tweet'].apply(fix_encoding)
df_clean_base_test["tweet"] = df_clean_base_test['tweet'].apply(fix_encoding)

#### 2. Datenbereinigung speziell für Textverarbeitung

##### Reihenfolge Datenbereinigung

Sinnvolle Reihenfolge für die Ausführung der einzelnen Bereinigungsschritte
1. Groß-/Kleinschreibung normalisieren
3. Die Textemojis auflösen und in emojis umwandeln
4. Abkürzungen/Slang auflösen

7. Umgang mit User-Handles
8. Umgang mit Hashtags (in Text belassen UND in gesonderte Spalte extrahieren)
9. Die zusammengeschriebene Wörter trennen
9. Emojis konvertieren (in Text konvertieren UND in gesonderte Spalte extrahieren)
10. Unnötige Zeichen entfernen
    a. Links
    b. html tags
    d. Punktuation
    e. Sonderzeichen
    f. Zeilenumbrüche
    g. Zahlen
    h. das Wort amp
11. Stemming / Lemmatization
12. Stopwords entfernen
13. Nochmal Duplikate prüfen & entfernen & leere Zeilen entfernen

Lemmatization: https://www.geeksforgeeks.org/python-lemmatization-approaches-with-examples/

In [ ]:
def clean_dataframe(base_df):
    df_cleaned = base_df.copy()
    df_cleaned['tweet_cleaned'] = df_cleaned['tweet']

    print("Start Cleaning")
   

    df_cleaned = to_lowercase(df_cleaned,'tweet_cleaned')
    print("Cleaning Step 2/19: expand_shortcuts")
    df_cleaned = replace_text_smileys(df_cleaned,'tweet_cleaned')
    print("Cleaning Step 2/19: replace_text_smileys")
    df_cleaned = expand_shortcuts(df_cleaned,'tweet_cleaned')
    print("Cleaning Step 4/19: handle_userhandles")
    df_cleaned = handle_userhandles(df_cleaned,'tweet_cleaned')
    print("Cleaning Step 5/19: handle_hashtags")
    df_cleaned = handle_hashtags(df_cleaned,'tweet_cleaned')
    print("Cleaning Step 1/19: segment_tweets")
   
    df_cleaned = segment_tweets(df_cleaned,'tweet_cleaned')
   
    print("Cleaning Step 6/19: extract_emojis")
    df_cleaned = extract_emojis(df_cleaned,'tweet_cleaned')
    print("Cleaning Step 7/19: replace_emojis")
    print("Cleaning Step 2/19: replace_text_smileys")
    df_cleaned = replace_emojis(df_cleaned,'tweet_cleaned')
    print("Cleaning Step 9/19: remove_url_from_tweet")
    df_cleaned = remove_url_from_tweet(df_cleaned,'tweet_cleaned')
    print("Cleaning Step 10/19: remove_punctuation")
    df_cleaned = remove_punctuation(df_cleaned,'tweet_cleaned')
    print("Cleaning Step 11/19: remove_special_characters")
    df_cleaned = remove_special_characters(df_cleaned, 'tweet_cleaned')
    print("Cleaning Step 12/19: remove_digis")
    df_cleaned = remove_digits(df_cleaned,'tweet_cleaned')
    print("Cleaning Step 13/19: remove_word_from_column: amp")
    df_cleaned = remove_word_from_column(df=df_cleaned, column_name="tweet_cleaned",word="amp")
    print("Cleaning Step 14/19: lemmatize")
    df_cleaned = lemmatize(df_cleaned,'tweet_cleaned')
    print("Cleaning Step 15/19: remove_stop_words")
    df_cleaned = remove_stop_words(df_cleaned,'tweet_cleaned')
    print("Cleaning Step 16/19: remove_most_frequent_words")
    df_cleaned = remove_most_frequent_words(df_cleaned,'tweet_cleaned')
    # print("Cleaning Step 17/19: remove_least_frequent_words")
    # df_cleaned = remove_least_frequent_words(df_cleaned,'tweet_cleaned')
    print("Cleaning Step 18/19: remove_duplicates")
    df_cleaned = remove_duplicates(df_cleaned,'tweet_cleaned')
    print("Cleaning Step 19/19: remove_nans")
    df_cleaned=remove_na_from_column(df=df_cleaned, column_name="tweet_cleaned")
    print("All Cleaning done")

    return df_cleaned


def clean_dataframe_test(base_df):
    df_cleaned = base_df.copy()
    df_cleaned['tweet_cleaned'] = df_cleaned['tweet']

    print("Start Cleaning")
   

    df_cleaned = to_lowercase(df_cleaned,'tweet_cleaned')
    print("Cleaning Step 2/19: expand_shortcuts")
    df_cleaned = replace_text_smileys(df_cleaned,'tweet_cleaned')
    print("Cleaning Step 2/19: replace_text_smileys")
    df_cleaned = expand_shortcuts(df_cleaned,'tweet_cleaned')
    print("Cleaning Step 4/19: handle_userhandles")
    df_cleaned = handle_userhandles(df_cleaned,'tweet_cleaned')
    print("Cleaning Step 5/19: handle_hashtags")
    df_cleaned = handle_hashtags(df_cleaned,'tweet_cleaned')
    print("Cleaning Step 1/19: segment_tweets")
   
    df_cleaned = segment_tweets(df_cleaned,'tweet_cleaned')
   
    print("Cleaning Step 6/19: extract_emojis")
    df_cleaned = extract_emojis(df_cleaned,'tweet_cleaned')
    print("Cleaning Step 7/19: replace_emojis")
    print("Cleaning Step 2/19: replace_text_smileys")
    df_cleaned = replace_emojis(df_cleaned,'tweet_cleaned')
    print("Cleaning Step 9/19: remove_url_from_tweet")
    df_cleaned = remove_url_from_tweet(df_cleaned,'tweet_cleaned')
    print("Cleaning Step 10/19: remove_punctuation")
    df_cleaned = remove_punctuation(df_cleaned,'tweet_cleaned')
    print("Cleaning Step 11/19: remove_special_characters")
    df_cleaned = remove_special_characters(df_cleaned, 'tweet_cleaned')
    print("Cleaning Step 12/19: remove_digis")
    df_cleaned = remove_digits(df_cleaned,'tweet_cleaned')
    print("Cleaning Step 13/19: remove_word_from_column: amp")
    df_cleaned = remove_word_from_column(df=df_cleaned, column_name="tweet_cleaned",word="amp")
    print("Cleaning Step 14/19: lemmatize")
    df_cleaned = lemmatize(df_cleaned,'tweet_cleaned')
    print("Cleaning Step 15/19: remove_stop_words")
    df_cleaned = remove_stop_words(df_cleaned,'tweet_cleaned')
    print("Cleaning Step 16/19: remove_most_frequent_words")
    df_cleaned = remove_most_frequent_words(df_cleaned,'tweet_cleaned')
    print("Cleaning Step 17/19: remove_least_frequent_words")
    df_cleaned = remove_least_frequent_words(df_cleaned,'tweet_cleaned')
    print("Cleaning Step 18/19: remove_duplicates")
    df_cleaned = remove_duplicates(df_cleaned,'tweet_cleaned')
    print("Cleaning Step 19/19: remove_nans")
    df_cleaned=remove_na_from_column(df=df_cleaned, column_name="tweet_cleaned")
    print("All Cleaning done")
    return df_cleaned

#### 2.1 Datenbereinigung Train Datensatz

In [18]:

df_cleaned = clean_dataframe(df_clean_base)

Start Cleaning
Cleaning Step 2/19: expand_shortcuts
Cleaning Step 2/19: replace_text_smileys
Cleaning Step 4/19: handle_userhandles
Cleaning Step 5/19: handle_hashtags
Cleaning Step 1/19: segment_tweets
Cleaning Step 6/19: extract_emojis
Cleaning Step 7/19: replace_emojis
Cleaning Step 2/19: replace_text_smileys
Cleaning Step 9/19: remove_url_from_tweet
Cleaning Step 10/19: remove_punctuation
Cleaning Step 11/19: remove_special_characters
Cleaning Step 12/19: remove_digis
Cleaning Step 13/19: remove_word_from_column: amp
Cleaning Step 14/19: lemmatize
Cleaning Step 15/19: remove_stop_words
Cleaning Step 16/19: remove_most_frequent_words
Cleaning Step 18/19: remove_duplicates
Cleaning Step 19/19: remove_nans
All Cleaning done


#### 2.2 Datenbereinigung Test Datensatz

In [17]:
df_cleaned_test = clean_dataframe_test(df_clean_base_test)

Start Cleaning
Cleaning Step 2/19: expand_shortcuts
Cleaning Step 2/19: replace_text_smileys
Cleaning Step 4/19: handle_userhandles
Cleaning Step 5/19: handle_hashtags
Cleaning Step 1/19: segment_tweets
Cleaning Step 6/19: extract_emojis
Cleaning Step 7/19: replace_emojis
Cleaning Step 2/19: replace_text_smileys
Cleaning Step 9/19: remove_url_from_tweet
Cleaning Step 10/19: remove_punctuation
Cleaning Step 11/19: remove_special_characters
Cleaning Step 12/19: remove_digis
Cleaning Step 13/19: remove_word_from_column: amp
Cleaning Step 14/19: lemmatize
Cleaning Step 15/19: remove_stop_words
Cleaning Step 16/19: remove_most_frequent_words
Cleaning Step 18/19: remove_duplicates
Cleaning Step 19/19: remove_nans
All Cleaning done


In [12]:
df_cleaned_check = df_cleaned[df_cleaned['tweet'].str.contains(":", regex=False)]

df_cleaned_check = df_cleaned_check[df_cleaned_check['user_handle']>= 1]
df_cleaned_check = df_cleaned_check[df_cleaned_check['hashtags'].apply(len) > 1]


df_cleaned_check = df_cleaned_check[df_cleaned_check['emojis'].apply(lambda x: len(x) > 0)]


df_cleaned_check.head(50)

,label,tweet,tweet_cleaned,user_handle,hashtags,emojis
id,,,,,,
14977,0,"@user #thanks for #sharing seattle sounders fc, #sunday :) #sharingiscaring (via ➪ grow followers 🆓)",thank share seattle sounder fc sunday smiley sharing care via rightwards arrow grow follower free button,1,"[#thanks, #sharing, #sunday, #sharingiscaring]","rightwards arrow,__FREE_button__"
28944,0,"@user #thanks for #sharing andrew jones, #sunday :) #sharingiscaring (via ➪ grow followers 🆓)",thank share andrew jones sunday smiley sharing care via rightwards arrow grow follower free button,1,"[#thanks, #sharing, #sunday, #sharingiscaring]","rightwards arrow,__FREE_button__"
7187,0,".@user on #periscope: rant!!! orlando, florida 😔😔😔 #hu",periscope rant orlando florida pensive face pensive pensive hu,1,"[#periscope, #hu]","__pensive_face__,__pensive_face__,__pensive_face__"
24117,0,okay @user i am literally going to scream. you are out of cold brew again!!!! is 6:30am how is this possible! #halp #why 😅☕️,okay literally go scream cold brew possible halp grinning face sweat hot beverage,1,"[#halp, #why]","__grinning_face_with_sweat__,__hot_beverage__"
23085,0,@user en directo en #periscope: morning at the #gym #workout #aerobics #success #ican #wekhan #love ❤❤❤❤❤❤❤💪💪💪🐩…,en directo en periscope morning gym work aerobics success khan love red heart red red red red red red flex bicep flex bicep flex bicep poodle …,1,"[#periscope, #gym, #workout, #aerobics, #success, #ican, #wekhan, #love]","__red_heart__,__red_heart__,__red_heart__,__red_heart__,__red_heart__,__red_heart__,__red_heart__,__flexed_biceps__,__flexed_biceps__,__flexed_biceps__,__poodle__"
29864,0,"😌 @user #21hrlater (from that moment of #zenserenity): #theworld, &amp; evrything in't, seems geared 2 make me #anxious 😡 &amp; 😞",relieve face hr later moment zen serenity world ev rything seems gear make anxious enraged disappointed,1,"[#21hrlater, #zenserenity, #theworld, #anxious]","__relieved_face__,__enraged_face__,__disappointed_face__"
9437,0,@user jts88's phoro: @user 😍 #love #sinkthepink #music #instagay #celebspot,jtss phoro smile face heart - eye love sink pink music insta gay celeb spot,2,"[#love, #sinkthepink, #music, #instagay, #celebspot]",__smiling_face_with_heart-eyes__
22134,0,"@user mood of the day: peace, love, happiness 💛👍🏻🌺 @user #love #life #hairstyle #tiger",mood day peace love happiness yellow heart thumb light skin tone hibiscus life hairstyle tiger,2,"[#love, #life, #hairstyle, #tiger]","__yellow_heart__,__thumbs_up__,__light_skin_tone__,__hibiscus__"
23633,0,"@user #thanks for #sharing security tube, #tuesday :) #sharingiscaring 🔹🆓grow followers ➪ 🆓🔹",thank share security tube tuesday smiley sharing care small blue diamond free button grow follower rightward arrow free button small blue diamond,1,"[#thanks, #sharing, #tuesday, #sharingiscaring]","__small_blue_diamond__,__FREE_button__,rightwards arrow,__FREE_button__,__small_blue_diamond__"


### Die Ergebnisse speichern

In [39]:

# train_cleaned abspeichern mit zusätzlichen Spalten
filepath_name = os.path.abspath(os.path.join(src_path, 'data/twitter_hate-speech/train_cleaned.csv'))
df_cleaned.to_csv(filepath_name)

# train_cleaned abspeichern ohne zusätzliche Spalten
df_cleaned_basic = df_cleaned[['label','tweet','tweet_cleaned']]
filepath_name = os.path.abspath(os.path.join(src_path, 'data/twitter_hate-speech/train_basic_cleaned.csv'))
df_cleaned_basic.to_csv(filepath_name)

In [40]:

# test_cleaned abspeichern mit zusätzlichen Spalten, ohne Duplikatenentfernung
filepath_name_test = os.path.abspath(os.path.join(src_path, 'data/twitter_hate-speech/test_cleaned.csv'))
df_cleaned_test.to_csv(filepath_name_test)

# test_cleaned abspeichern ohne zusätzliche Spalten, ohne Duplikatenentfernung
df_cleaned_test_new =  df_cleaned_test[['tweet','tweet_cleaned']]
filepath_name_test = os.path.abspath(os.path.join(src_path, 'data/twitter_hate-speech/test_basic_cleaned.csv'))
df_cleaned_test_new.to_csv(filepath_name_test)